In [1]:
import numpy as np
import pandas as pd
from numba import jit
import cv2
import math
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

### Filter

In [2]:
@jit
def cal_Filter(x,y,radiusInside,radiusOutside,r,Filter):
    if radiusInside <= r and r <= radiusOutside:
        Filter[y][x] = 0
    else:
        Filter[y][x] = -1000000
    
    return Filter

### dilation

In [3]:
@jit#(i4(i4,i4,i4,i4,u1[:,:],f8[:,:],u4,u4,u4,u4,i4))
def cal_dilation(u,v,xu,yv,f,g,f_x,f_y,g_x,g_y,maxValue):
    if 0 <= xu and xu<=f_x-1 and 0 <= yv and yv <= f_y-1: 
        if 0 <= u+int(g_x/2) and u+int(g_x/2) <= g_x and 0 <= v+int(g_y/2) and v+int(g_y/2) <= g_y:
            #print(xu, yv, 'f:',f[yv][xu],'g:',g[v+int(g_y/2)][u+int(g_x/2)])
            value = f[yv][xu] + g[v+int(g_y/2)][u+int(g_x/2)]
            maxValue = max(maxValue, value)
    
    return maxValue

### diaOut

In [4]:
@jit#(u4[:,:](u4[:],u4,u4,u4))
def cal_diaOut(diaOut,x,y,maxValue):
    diaOut[y][x] = maxValue
    
    return diaOut

### create

In [5]:
@jit#(u4[:,:](u4,u4))
def createFilter(radiusOutside, radiusInside):
    Filter = np.zeros((radiusOutside*2+1, radiusOutside*2+1))
    Filter_x, Filter_y = len(Filter[0]), len(Filter)
    for y in range(Filter_y):
        for x in range(Filter_x):
            u = x - Filter_x/2
            v = y - Filter_y/2
            r = math.sqrt(u*u+v*v)
            
            Filter = cal_Filter(x,y,radiusInside,radiusOutside,r,Filter)
    
    return Filter

### dilation def

In [6]:
@jit#(u4[:,:](u4[:],u4[:]))
def dilation(f, g):
    f_x, f_y = len(f[0]), len(f)
    g_x, g_y = len(g[0]), len(g)
#    print(f_x, f_y)
#    print(g_x, g_y)
#    print(int(g_x/2),int(g_y/2))
    diaOut = np.zeros((f_y, f_x))
    for y in range(f_y):
        for x in range(f_x):
            maxValue = -1000000
            for v in range(g_y):
                v = v - int(g_y/2)
                for u in range(g_x):
                    u = u - int(g_x/2)
                    xu = x + u
                    yv = y + v
                    maxValue = cal_dilation(u,v,xu,yv,f,g,f_x,f_y,g_x,g_y,maxValue)
            diaOut = cal_diaOut(diaOut,x,y,maxValue)
    
    return diaOut

### 画像差

In [7]:
def cal_diff(x, y, ringImg, diskImg):
    value = diskImg[y][x] - ringImg[y][x]
    outImg[y][x] = value   
            
    return outImg

### 要素消す

In [8]:
def get_unique_list(seq):
    seen = []
    return [x for x in seq if x not in seen and not seen.append(x)]

### CNN用設定

In [9]:
import tensorflow as tf
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD

Using TensorFlow backend.


In [10]:
#重みの読み込み
weights_path = 'weight/ex12_100_area2回目.h5'

In [11]:
#CNNパラメータ設定
classes = 2
class_name = ['defect','other']
ker = cv2.INTER_LINEAR
input_size = 227


model = Sequential()

#第一層Cov1
model.add(
        Conv2D(
                filters = 96,
                kernel_size = (11,11),
                strides = (4,4),
                padding = 'valid',
                input_shape = (227,227,1),
                activation = 'relu'
                ))
#正則化
#model.add(BatchNormalization())

#第二層Pool1
model.add(
        MaxPooling2D(pool_size=(3,3),
                     strides = (2,2),
                     padding = 'valid')
        )
        
#第三層Cov2
model.add(
        Conv2D(
                filters = 128,
                kernel_size = (5,5),
                strides = (1,1),
                padding = 'valid',
                activation = 'relu'
                ))

#正則化
#model.add(BatchNormalization())

#第四層Pool2
model.add(
        MaxPooling2D(pool_size=(3,3),
                     strides = (2,2),
                     padding = 'valid')
        )
        
#第五層Cov3-1
model.add(
        Conv2D(
                filters = 256,
                kernel_size = (3,3),
                strides = (1,1),
                padding = 'same',
                activation = 'relu'
                ))

#第六層Cov3-2
model.add(
        Conv2D(
                filters = 256,
                kernel_size = (3,3),
                strides = (1,1),
                padding = 'same',
                activation = 'relu'
                ))

#第七層Cov3-3
model.add(
        Conv2D(
                filters = 128,
                kernel_size = (3,3),
                strides = (1,1),
                padding = 'same',
                activation = 'relu'
                ))

#第八層Pool3
model.add(
        MaxPooling2D(pool_size=(3,3),
                     strides = (2,2),
                     padding = 'valid')
        )

#Flatten層
model.add(Flatten())
        
#第九層FC1
model.add(
        Dense(1000)
        )


#第十層FC2
model.add(
        Dense(256)
        )    
 
#出力
model.add(
        Dense(classes,
              activation='softmax'))

#Sequentialオブジェクトのコンパイル
model.compile(
        loss = 'categorical_crossentropy',
        optimizer = SGD(lr=0.001,momentum=0.9,decay=0.00005),
        metrics = ['accuracy']
        )

#モデルのサマリーを表示
model.summary()

#重みを読み込む
model.load_weights(weights_path)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        11712     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       307328    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 256)       590080    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 11, 11, 128)      

In [12]:
#予測関数
from keras. applications.vgg16 import preprocess_input

def predict(img):
    img = cv2.resize(img,(227,227),interpolation=ker)
    x = np.array(img, dtype=np.float32)
    x = x/255.
    x = x[None,...]
    x = preprocess_input(x)
    x = x.reshape(1,input_size,input_size,1)
    pred = model.predict(x,verbose = 0)
    pred_label = class_name[np.argmax(pred)]
    
    return pred_label

### SegNet用設定

In [13]:
def mc_output(model, img, width, height, mc_num, cls_num): 
    preds = np.zeros((width,height,cls_num))
    for i in range(mc_num):
        out = model.predict_segmentation(inp=img)
        preds = preds+np.eye(cls_num)[out]
    return preds

In [14]:
def cls2img(pred_img,cls_n):
    shape = pred_img.shape
    max_cls = np.argmax(pred_img, axis = 2)
    out = np.zeros((shape[0], shape[1], 3))
    for i in range(shape[0]):
        for j in range(shape[1]):
            out[i, j] = cls_n[max_cls[i, j]]
        
    return out

In [15]:
import keras_segmentation2.models.segnet
from collections import OrderedDict

### 各種パラメータ設定

In [16]:
dil = 3
kernel = 2
kernel_size = np.ones((kernel,kernel),np.uint8)
#リングの内径
ring = 1
#ディスク半径
disk = 25

In [17]:
dil_rev = 2
kernel_rev = 2
kernel_size_rev = np.ones((kernel_rev,kernel_rev),np.uint8)
ring_rev = 1
disk_rev = 5

In [18]:
import glob
import os

input_data = 'check_data'

all_data = glob.glob('{}/*'.format(input_data))
all_name = [os.path.basename(p) for p in all_data
       if os.path.isfile(p)]

### 実装

In [20]:
import time
# 処理前の時刻
t1 = time.time() 

#欠陥数出力リスト
defect_num = []

#input画像準備

count_name = 0

for img_path in all_data: 
    name = all_name[count_name]

    
    #画像読み込み
    img_color = cv2.imread(img_path)
    img_contour = cv2.imread(img_path)
    img_quoit = cv2.imread(img_path)
    img_CNN = cv2.imread(img_path)
    img_CNN_cut = cv2.imread(img_path)
    
    color = cv2.imread(img_path)
    #グレースケール
    img = cv2.imread(img_path,0)
    #height,width
    h,w = img.shape[:2]
    
    #dilation処理
    for i in range(dil):
        #最初はimg読み込み
        if i == 0:
            d_img = cv2.dilate(img,kernel_size,iterations = 1)
        #2回目以降はd_imgを読み込み
        else:
            d_img = cv2.dilate(d_img,kernel_size,iterations = 1)
    d_img = cv2.resize(d_img,(480, 270))
    
    
    #反転用
    img_rev = cv2.bitwise_not(img)
    for i in range(dil_rev):
        if i == 0:
            d_img_rev = cv2.dilate(img_rev,kernel_size_rev,iterations = 1)
        else:
            d_img_rev = cv2.dilate(d_img_rev,kernel_size_rev,iterations = 1)
    d_img_rev = cv2.resize(d_img_rev,(480, 270))
    
    
    ### quoit実装
    
    
    #初期値設定
    R1 = disk #Disk外径
    R2 = ring #Ring内径
    R3 = disk #Ring外径

    #画像の読み込み
    inImg = d_img
    inImgSM = inImg

    #フィルタ作成
    diskFilter = createFilter(R1, 0)
    ringFilter = createFilter(R3, R2)
    #print('フィルタは起動している')

    #Dilation画像を求める
    diskImg = dilation(inImgSM, diskFilter)
    ringImg = dilation(inImgSM, ringFilter)
    #差分を計算し、出力画像用の配列を用意
    #出力用の画像を用意
    img_x, img_y = len(d_img[0]), len(d_img)
    outImg = np.zeros((img_y, img_x))
    #両画像の差分を求める
    for y in range(img_y):
        for x in range(img_x):
            outImg = cal_diff(x, y, ringImg, diskImg)
    outImg = cv2.resize(outImg,(w, h))
    #輪郭取得&重心取得
    #グレースケール画像
    imageGRAY = outImg
    blur = imageGRAY
    #閾値で二値化する
    threshold = 0.01
    ret1, th1 = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
    #データ型を直す
    th1 = th1.astype(np.uint8)

    #輪郭を抽出する
    contours, _ = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    dic = []
    for i in contours :
        x = 0
        y = 0
        for n in i :
            x += n[0][0]
            y += n[0][1]
        x = int(x/len(i))
        y = int(y/len(i))
        dic.append([[y-50,y+50,x-50,x+50],i])

    #矩形と集合のリスト化
    dic = np.array(dic)
        
        
    ### 反転用

    #初期値設定
    R1 = disk_rev #Disk外径
    R2 = ring_rev #Ring内径
    R3 = disk_rev #Ring外径
    #画像の読み込み
    inImg = d_img_rev
    inImgSM = inImg

    #フィルタ作成
    diskFilter = createFilter(R1, 0)
    ringFilter = createFilter(R3, R2)
    #print('フィルタは起動している')

    #Dilation画像を求める
    diskImg = dilation(inImgSM, diskFilter)
    ringImg = dilation(inImgSM, ringFilter)
    #差分を計算し、出力画像用の配列を用意
    #出力用の画像を用意
    img_x, img_y = len(d_img[0]), len(d_img)
    outImg = np.zeros((img_y, img_x))
    #両画像の差分を求める
    for y in range(img_y):
        for x in range(img_x):
            outImg = cal_diff(x, y, ringImg, diskImg)
    outImg = cv2.resize(outImg,(w, h))
    #輪郭取得&重心取得
    #グレースケール画像
    imageGRAY = outImg
    blur = imageGRAY
    #閾値で二値化する
    threshold = 0.01
    ret1, th1 = cv2.threshold(blur, threshold, 255, cv2.THRESH_BINARY)
    #データ型を直す
    th1 = th1.astype(np.uint8)

    #輪郭を抽出する
    contours, _ = cv2.findContours(th1, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    dic_rev = []
    for i in contours :
        x = 0
        y = 0
        for n in i :
            x += n[0][0]
            y += n[0][1]
        x = int(x/len(i))
        y = int(y/len(i))
        dic_rev.append([[y-50,y+50,x-50,x+50],i])
    #矩形と集合のリスト化
    dic_rev = np.array(dic_rev)
    
    
    #領域と輪郭を配列にまとめる

    def_loc = []
    def_con = []

    for loc,con in dic:
        def_loc.append(loc)
        def_con.append(con)
    for loc,con in dic_rev:
        def_loc.append(loc)
        def_con.append(con)
        
    
    #被りがあったら削除

    def_loc = get_unique_list(def_loc)
    
    #quoit絞り込み
    print("quoit_{}_抽出された欠陥の数:".format(name), len(def_loc))
    
    
    #Contour画像
    cv2.drawContours(img_contour, contours, -1, color=(255, 0, 0), thickness=1)
    cv2.imwrite('output_data/Contour/Contour_{}'.format(name), img_contour)
    
    #欠陥領域描写

    for loc in def_loc:
        if loc[0] < 0:
            loc[0] = 0
        if loc[2] < 0:
            loc[2] = 0

    for i, loc in enumerate(def_loc):
        if i == 0:
            #検出領域を四角で囲んで保存
            result = img_quoit
            result1 = cv2.rectangle(result,(loc[2],loc[0]), (loc[3],loc[1]), (255, 0, 0), 2)

        if i != 0:
            #検出領域を四角で囲んで保存
            result = result1
            result1 = cv2.rectangle(result, (loc[2], loc[0]), (loc[3], loc[1]), (255, 0, 0), 2)
            
    #保存
    cv2.imwrite('output_data/quoit/quoit_{}'.format(name), result1) 
    
    
    #CNN適用

    pred_loc = []

    for loc in def_loc:
        
        cut = img[loc[0]:loc[1],loc[2]:loc[3]]
        #print(loc,cut)
        pred_label = predict(cut)
        #print(pred_label)
        if pred_label == 'defect':
            pred_loc.append(loc)
            
    #CNN絞り込み
    print("CNN_{}_抽出された欠陥の数:".format(name), len(pred_loc))
    
    
    for i, loc in enumerate(pred_loc):
        if i == 0:
            #検出領域を四角で囲んで保存
            cnn_result = img_CNN
            cnn_result1 = cv2.rectangle(cnn_result,(loc[2],loc[0]), (loc[3],loc[1]), (255, 0, 0), 2)
        
        if i != 0:
            #検出領域を四角で囲んで保存
            cnn_result = cnn_result1
            cnn_result1 = cv2.rectangle(cnn_result, (loc[2], loc[0]), (loc[3], loc[1]), (255, 0, 0), 2)
            
    #保存
    cv2.imwrite('output_data/CNN/CNN_{}'.format(name), cnn_result1)
    
    #CNNcut用のdir作成
    dir_path = 'output_data/CNN_cut/{}'.format(name[0:-4])
    os.makedirs(dir_path, exist_ok=True)
    
    #CNNcut保存
    cut_n = 0
    for loc in pred_loc:

        CNN_cut = img_CNN_cut[loc[0]:loc[1], loc[2]:loc[3]]

        cv2.imwrite('{}/CNN_cut_{}_{}'.format(dir_path, cut_n, name), CNN_cut)

        cut_n += 1
    
    
    #CNN_cut座標
    df = pd.DataFrame(pred_loc)
    df.columns = ['top', 'bottom', 'left', 'right']
    
    #書き出し
    df.to_csv('{}/{}_CNN_cut_cordinates.csv'.format(dir_path, name[0:-4]), mode='w')
    
    
    
    ###SegNet
    
    cls = OrderedDict() #順序づき辞書
    #色と欠点種別の対応
    cls = {"背景":[0,0,0], "欠陥":[255,0,0]}
    #色とインデックスの対応
    cls_n = [[0,0,0],[255,0,0]]  

    my_bayesSeg = keras_segmentation2.models.segnet.my_bayes_segnet(n_classes = 2,
                                                                input_height = 384,
                                                                input_width = 384,
                                                                encoder_level = 3,
                                                                dropout_rate = 0.02)
    my_bayesSeg.load_weights('weight/seg_weight01_20_22_23.h5')

    count = 0
    
    #SegNet_cut用のdir作成
    
    dir_path1 = 'output_data/SegNet_cut/{}'.format(name[0:-4])
    os.makedirs(dir_path1, exist_ok=True)
    
    cut_n = 0
    
    #SegNet座標リスト
    SegNet_list = []
    SegNet_cut_list = []
    
    blank = np.zeros((2160,3840,3))
    for loc in pred_loc:
        
        SegNet_check = 0
        
        
        try:
            #矩形情報から図形の中心を求める
            y = loc[0]+50
            x = loc[2]+50

            inp = color[y-192:y+192,x-192:x+192]
            inp = cv2.resize(inp,(384,384))
            out = mc_output(my_bayesSeg, inp,192,192,1,2)
            out = cls2img(out, cls_n)
            #cv2.imwrite('mask_'+str(count)+'.png',out)
            #print(out.shape)
            out = cv2.resize(out,(384,384))



            #取れた領域の中央100ピクセル四方で囲まれた領域に出た出力のみオリジナル画像に描画していく

            for j in range(142,243):
                for i in range(142,243):
                    #print(i,j)
                    if out[j][i][0]+out[j][i][1]+out[j][i][2] != 0:
                        img_color[y-192+j][x-192+i] = [0,0,255]
                        blank[y-192+j][x-192+i] = [0,0,255]
                        SegNet_check += 1
                        SegNet_list.append([cut_n, (y-192+j), (x-192+i)])
                        

        except:
            #print(x,y)

            continue

        img_color = np.array(img_color)
        mask = np.array(blank)
        
        
        #SegNet_cut保存
        if SegNet_check > 0:
            SegNet_cut_list.append([loc[0], loc[1], loc[2], loc[3]])
            SegNet_cut = img_color[loc[0]:loc[1], loc[2]:loc[3]]
            cv2.imwrite('{}/SegNet_cut_{}_{}'.format(dir_path1, cut_n, name), SegNet_cut)
            cut_n += 1
          
        
        count += 1
    
    #SegNet_list_csv, SegNet_cut_list_csv
    df = pd.DataFrame(SegNet_list)
    df.columns = ['cut_number', 'y', 'x']
    
    df1 = pd.DataFrame(SegNet_cut_list)
    df1.columns = ['top', 'bottom', 'left', 'right']
    
    #書き出し
    df.to_csv('{}/{}_SegNet.csv'.format(dir_path1, name[0:-4]), mode='w')
    df1.to_csv('{}/{}_SegNet_cut_cordinates.csv'.format(dir_path1, name[0:-4]), mode='w')
    
    img_color = np.array(img_color)

    cv2.imwrite('output_data/SegNet/SegNet_{}'.format(name),img_color)
    cv2.imwrite('output_data/SegNet/mask_{}'.format(name),mask) 
    
    count_name += 1
    
    defect_num.append([name, len(def_loc), len(pred_loc), len(SegNet_cut_list)])
    

#defect_num_csv    
df = pd.DataFrame(defect_num)
df.columns = ['画像ファイル名', 'quoit', 'CNN', 'SegNet']
df.to_csv('output_data/defect_num_{}.csv'.format(input_data))

# 処理後の時刻
t2 = time.time()
 
# 経過時間を表示
elapsed_time = t2-t1
print(f"経過時間：{elapsed_time}")

quoit_blue1-1_187.png_抽出された欠陥の数: 978
CNN_blue1-1_187.png_抽出された欠陥の数: 32
quoit_black1-1_310.png_抽出された欠陥の数: 1109
CNN_black1-1_310.png_抽出された欠陥の数: 82
quoit_black1-3_741.png_抽出された欠陥の数: 998
CNN_black1-3_741.png_抽出された欠陥の数: 54
quoit_blue1-2_328.png_抽出された欠陥の数: 1018
CNN_blue1-2_328.png_抽出された欠陥の数: 42
経過時間：282.66994547843933
